In [ ]:
import numpy as np
import sqlite3
from dataclasses import dataclass
from sqlite3 import Connection, Cursor

from typing import Callable, List, Tuple

from model import Area, Route, RouteRating, RouteReview, RouteTick

In [168]:
def get_users(
  cursor: Cursor, limit_function: Callable[[int], int]) -> List[Tuple[int, int]]:
    '''
    Return a list of tuples of form (user_id, route_count). The number of total
    users is passed to the limit function, which is passed to SQL as the limit of
    users to return.
    '''

    cursor.execute('SELECT COUNT(DISTINCT(user_id)) FROM reviews')
    total_user_count = cursor.fetchone()[0]

    limit = limit_function(total_user_count)

    cursor.execute('''SELECT user_id, COUNT(*) AS review_count
                   FROM reviews
                   GROUP BY user_id
                   LIMIT ?;''', [limit])
    
    return cursor.fetchall()


In [193]:
def normalize_review(review):
    if review is None:
        return 0

    return 1 if review[0] >= 3 else -1


def get_climbs_for_user(cursor: Cursor, user_id: int) -> List[Tuple[int, int]]:
    '''
    Return a list of tuples of the form (route_id, score) for a particular user.
    '''

    cursor.execute('''SELECT route_id, score
                   FROM reviews
                   WHERE user_id = ?;''', [int(user_id)])

    rows = cursor.fetchall()

    return np.array(rows)


def get_users_for_climb(cursor: Cursor, route_id: int) -> List[int]:
    '''
    Return a list of users that have left a review on a route.
    '''

    cursor.execute('''SELECT user_id
                   FROM reviews
                   WHERE route_id = ?;''', [int(route_id)])

    return np.array(cursor.fetchall())[:, 0]


def get_route_vector_for_users(
  cursor: Cursor, a: int, b: int):
    '''
    Return an array with many entries. Each entry
    '''
    
    cursor.execute('''SELECT a.score, b.score
                   FROM reviews a
                   JOIN reviews b
                   ON 1=1
                    AND a.route_id = b.route_id
                    AND a.user_id != b.user_id
                   WHERE 1=1
                    AND a.user_id = ?
                    AND b.user_id = ?
                   LIMIT 10;''', [int(a), int(b)])

    rows = cursor.fetchall()

    return np.array(rows)

# Calls

In [170]:
conn = sqlite3.connect('databasev2.db')
cursor = conn.cursor()

In [171]:
# (user_id, review_count)
user_routecount_tuples = np.array(
    sorted(
        get_users(cursor, lambda n: int(n * 0.25)), 
        key=lambda x: x[1], reverse=True))

users = user_routecount_tuples[:, 0]

user_routecount_tuples

array([[    10788,      6618],
       [    14537,      5157],
       [    10232,      4948],
       ...,
       [109498998,         1],
       [109499247,         1],
       [109499485,         1]], shape=(25590, 2))

In [172]:
test_user = users[10]
# (route_id, review)
test_user_reviews = get_climbs_for_user(cursor, test_user)
# (route_id)
test_routes = np.array(test_user_reviews[:, 0])

# We are going to see how the recommendation works for this one
test_route = 105717289
# Index of the route in the vectors
test_index = int(np.where(test_routes == test_route)[0][0])
test_reviews = test_user_reviews[:, 1]
test_review = test_reviews[test_index]

We need to collect vectors for multiple other users. Those vectors should be of the shape (review) and should have the same order as the test_user.

In [209]:
vectors = []

test_users = get_users_for_climb(cursor, test_route)

for i, user in enumerate(test_users[test_users != test_user][:30]):
    print(i)
    vectors.append(get_route_vector_for_users(cursor, test_user, user))

'''
routes = test_routes[:30]

def job(user_id):
    print(user_id)
    conn = sqlite3.connect('databasev2.db')
    return get_route_vector_for_user(conn.cursor(), routes, user_id)


with Pool(10) as pool:
    vectors = pool.map(job, list(map(int, users[users != test_user][:30])))
'''

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


"\nroutes = test_routes[:30]\n\ndef job(user_id):\n    print(user_id)\n    conn = sqlite3.connect('databasev2.db')\n    return get_route_vector_for_user(conn.cursor(), routes, user_id)\n\n\nwith Pool(10) as pool:\n    vectors = pool.map(job, list(map(int, users[users != test_user][:30])))\n"

In [210]:
vectors

[array([[4, 5],
        [4, 5],
        [5, 5],
        [5, 5],
        [5, 5],
        [4, 5],
        [4, 4],
        [4, 4],
        [4, 4],
        [5, 5]]),
 array([[4, 5],
        [4, 5],
        [5, 5],
        [5, 4]]),
 array([[4, 5],
        [4, 5],
        [3, 5],
        [3, 5],
        [5, 4],
        [5, 4],
        [4, 5],
        [4, 5],
        [5, 5],
        [4, 5]]),
 array([[4, 5],
        [4, 5],
        [4, 4],
        [5, 5],
        [4, 3],
        [2, 2]]),
 array([[4, 5],
        [5, 5],
        [3, 3],
        [4, 4],
        [4, 4],
        [2, 2],
        [3, 3],
        [3, 3],
        [4, 4],
        [4, 4]]),
 array([[4, 5],
        [4, 5],
        [5, 5],
        [4, 5],
        [4, 4],
        [5, 5],
        [5, 4],
        [5, 4],
        [5, 4],
        [5, 4]]),
 array([[4, 5],
        [5, 5],
        [4, 4],
        [4, 5],
        [3, 4],
        [4, 4],
        [3, 3],
        [4, 4]]),
 array([[4, 5],
        [3, 5],
        [5, 5],
        [5

In [ ]:
@dataclass
class CosineSimilarity:
    pass


def cosine_sim(a, b):
    dot = np.dot(a, b)
    magnitude_a = np.linalg.norm(a)
    magnitude_b = np.linalg.norm(b)

    if magnitude_a * magnitude_b == 0:
        return 0

    return dot / (magnitude_a * magnitude_b)


for matrix in vectors:
    if len(matrix) == 0:
        continue
    print(cosine_sim(matrix[:, 0], matrix[:, 1]))

0.9949078734138886
0.9839914013827435
0.9733467146850807
0.9863103496732771
0.9969527608177988
0.9835204696632591
0.9931649513162263
0.9866556154697232
0.9938586931957764
1.0
0.9845264800162247
0.9948497511671098
0.9888264649460885
0.9927741970308562
0.9909232001987206
0.990774623617847
0.9574271077563381
0.9589180631998816
0.9860791939485092
0.989558818715312
0.9826595893986433
0.988021485085578
0.9822191694525506
0.9877027133661868
0.9802861627917436
0.9935690651280802
0.9831353843426084
0.9907569103664061
0.9944903161976939
0.9939379672106393
